In [1]:
import xarray as xr
import glob
import numpy as np

In [2]:
import os

In [3]:
# get sorted list of available files of flagged and non-flagged sensors
fn_list_flagged = glob.glob(
    "/pd/data/CML/data/reference/anomaly_flags/combined_4_experts/flags_20_cml_2019_*.nc"
)
fn_list = glob.glob(
    "/pd/data/CML/data/processed/proc2021.001/proc_hess_amt/proc_cnn_gapstandard_2019_*.nc"
)
fn_list_flagged.sort()
fn_list.sort()

In [4]:
fn_list

['/pd/data/CML/data/processed/proc2021.001/proc_hess_amt/proc_cnn_gapstandard_2019_01.nc',
 '/pd/data/CML/data/processed/proc2021.001/proc_hess_amt/proc_cnn_gapstandard_2019_02.nc',
 '/pd/data/CML/data/processed/proc2021.001/proc_hess_amt/proc_cnn_gapstandard_2019_03.nc',
 '/pd/data/CML/data/processed/proc2021.001/proc_hess_amt/proc_cnn_gapstandard_2019_04.nc',
 '/pd/data/CML/data/processed/proc2021.001/proc_hess_amt/proc_cnn_gapstandard_2019_05.nc',
 '/pd/data/CML/data/processed/proc2021.001/proc_hess_amt/proc_cnn_gapstandard_2019_06.nc',
 '/pd/data/CML/data/processed/proc2021.001/proc_hess_amt/proc_cnn_gapstandard_2019_07.nc',
 '/pd/data/CML/data/processed/proc2021.001/proc_hess_amt/proc_cnn_gapstandard_2019_08.nc',
 '/pd/data/CML/data/processed/proc2021.001/proc_hess_amt/proc_cnn_gapstandard_2019_09.nc',
 '/pd/data/CML/data/processed/proc2021.001/proc_hess_amt/proc_cnn_gapstandard_2019_10.nc',
 '/pd/data/CML/data/processed/proc2021.001/proc_hess_amt/proc_cnn_gapstandard_2019_11.nc',

In [5]:
fn_list_flagged

['/pd/data/CML/data/reference/anomaly_flags/combined_4_experts/flags_20_cml_2019_03.nc',
 '/pd/data/CML/data/reference/anomaly_flags/combined_4_experts/flags_20_cml_2019_05.nc',
 '/pd/data/CML/data/reference/anomaly_flags/combined_4_experts/flags_20_cml_2019_07.nc']

# load data from archive

In [6]:
# load non-flagged data (all sensors)
ds = xr.open_dataset(fn_list[6])

In [7]:
# load flagged data (20 sensors)
ds_f = xr.open_dataset(fn_list_flagged[2])


# Processing steps

In [8]:
# drop flagged sensor ids from all sensors and duplicate time steps
ds_nf = ds.drop_sel({"cml_id": ds_f.cml_id.values}).drop_duplicates(
    dim="time", keep="first"
)
# concatenate flagged and non-flagged sensors. adds missing values in flags for non-flagged sensors
ds_m = xr.concat([ds_f, ds_nf], dim="cml_id")

In [9]:


# create variable that tracks which cmls were flagged by experts
ds_m["flagged"] = (
    "cml_id",
    ((~np.isnan(ds_m.Jump)).sum(dim="time").sum(dim="expert") > 0).values,
)
# split txrx into two variables along channel_id and rename to TL according to naming convention
ds_m["TL_1"] = ["cml_id", "time"], ds_m.txrx.isel(channel_id=0).values
ds_m["TL_2"] = ["cml_id", "time"], ds_m.txrx.isel(channel_id=1).values
# split frequency and polarization coords along channel_id dim
ds_m["frequency_1"] = "cml_id", ds_m.frequency.isel(channel_id=0).values
ds_m["frequency_2"] = "cml_id", ds_m.frequency.isel(channel_id=1).values
ds_m["polarization_1"] = "cml_id", ds_m.polarization.isel(channel_id=0).values
ds_m["polarization_2"] = "cml_id", ds_m.polarization.isel(channel_id=1).values
ds_m = ds_m.assign_coords(
    {
        "frequency_1": ("cml_id", ds_m.frequency.isel(channel_id=0).values),
        "frequency_2": ("cml_id", ds_m.frequency.isel(channel_id=1).values),
        "polarization_1": ("cml_id", ds_m.polarization.isel(channel_id=0).values),
        "polarization_2": ("cml_id", ds_m.polarization.isel(channel_id=1).values),
    }
)
# drop irrelevant variables based on which ones we want to keep
current_vars = list(ds_m.keys())
keep_vars = ["TL_1", "TL_2", "Jump", "Dew", "Fluctuation", "Unknown anomaly", "flagged"]
ds_m = ds_m.drop(list(set(current_vars) - set(keep_vars)))
# drop channel_id dimension
ds_m = ds_m.drop_dims(["channel_id"])
# rename dimension
ds_m = ds_m.rename_dims({"cml_id": "sensor_id"})
# rename coordinate var
ds_m = ds_m.rename_vars({"cml_id": "sensor_id"})


/tmp/ipykernel_1533380/14851095.py:25: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  ds_m = ds_m.drop(list(set(current_vars) - set(keep_vars)))


# Inspect result

In [10]:
ds_m

<xarray.Dataset> Size: 14GB
Dimensions:           (expert: 4, sensor_id: 3904, time: 44641)
Coordinates:
  * time              (time) datetime64[ns] 357kB 2019-07-01 ... 2019-08-01
  * sensor_id         (sensor_id) <U17 265kB 'BY4168_2_BY4036_4' ... 'SY8534_...
    length            (sensor_id) float64 31kB 12.62 11.39 12.43 ... 2.791 4.994
    site_a_latitude   (sensor_id) float64 31kB 51.42 51.33 51.62 ... 49.0 48.64
    site_a_longitude  (sensor_id) float64 31kB 13.94 8.152 8.165 ... 8.559 9.523
    site_b_latitude   (sensor_id) float64 31kB 51.38 51.39 51.66 ... 49.01 48.64
    site_b_longitude  (sensor_id) float64 31kB 13.77 8.286 8.334 ... 8.527 9.455
    frequency_1       (sensor_id) float64 31kB 1.814e+10 1.82e+10 ... 2.497e+10
    frequency_2       (sensor_id) float64 31kB 1.915e+10 1.92e+10 ... 2.598e+10
    polarization_1    (sensor_id) <U1 16kB 'V' 'V' 'V' 'V' ... 'V' 'V' 'V' 'V'
    polarization_2    (sensor_id) <U1 16kB 'V' 'V' 'V' 'V' ... 'V' 'V' 'V' 'V'
Dimensions without coordinates: expert
Data variables:
    Jump              (expert, sensor_id, time) float32 3GB 0.0 0.0 ... nan nan
    Dew               (expert, sensor_id, time) float32 3GB 0.0 0.0 ... nan nan
    Fluctuation       (expert, sensor_id, time) float32 3GB 0.0 0.0 ... nan nan
    Unknown anomaly   (expert, sensor_id, time) float32 3GB 0.0 0.0 ... nan nan
    flagged           (sensor_id) bool 4kB True True True ... False False False
    TL_1              (sensor_id, time) float64 1GB 70.2 68.9 70.2 ... 59.2 nan
    TL_2              (sensor_id, time) float64 1GB 71.2 70.2 70.9 ... 58.0 nan

# select example subset

In [11]:
non_flagged_example_cmls =  ['DO0202_2_DO6017_4', 'DO0206_2_DO6017_3',
       'DO0341_2_DO1665_4', 'DO0455_2_DO6017_4', 'DO0787_2_DO2382_2',
       'DO0951_2_DO1665_2', 'DO1689_2_DO0255_4', 'DO1744_2_DO0255_2',
       'DO1893_2_DO6017_3', 'DO1932_2_HY1317_4', 'DO2118_2_DO6038_3',
       'DO2796_2_DO6017_2', 'DO2867_2_DO6017_6', 'DO2974_2_DO0214_2',
       'DO3109_2_DO1893_3', 'DO3623_2_DO6017_2', 'DO5416_2_DO6075_2',
       'DO5533_2_DO0951_3', 'DO6001_2_DO6017_4', 'DO6066_2_DO6017_5',
       'DO6067_2_DO6017_3', 'HY0277_2_HY1317_3']

In [97]:
flagged_example_cml = 'DO1695_2_DO6017_2'
example_time = slice('2019-07-02T00:00:00.000000000','2019-07-29T23:59:00.000000000')
example_cmls = list(non_flagged_example_cmls)
example_cmls.append(flagged_example_cml)
print(example_cmls)

['DO0202_2_DO6017_4', 'DO0206_2_DO6017_3', 'DO0341_2_DO1665_4', 'DO0455_2_DO6017_4', 'DO0787_2_DO2382_2', 'DO0951_2_DO1665_2', 'DO1689_2_DO0255_4', 'DO1744_2_DO0255_2', 'DO1893_2_DO6017_3', 'DO1932_2_HY1317_4', 'DO2118_2_DO6038_3', 'DO2796_2_DO6017_2', 'DO2867_2_DO6017_6', 'DO2974_2_DO0214_2', 'DO3109_2_DO1893_3', 'DO3623_2_DO6017_2', 'DO5416_2_DO6075_2', 'DO5533_2_DO0951_3', 'DO6001_2_DO6017_4', 'DO6066_2_DO6017_5', 'DO6067_2_DO6017_3', 'HY0277_2_HY1317_3', 'DO1695_2_DO6017_2']


In [98]:
ds_e = ds_m.sel({
    'sensor_id': example_cmls,
    'time': example_time,
})

In [99]:
ds_e.nbytes/1e6

74.514419

In [100]:
ds_e

<xarray.Dataset> Size: 75MB
Dimensions:           (expert: 4, sensor_id: 23, time: 40320)
Coordinates:
  * time              (time) datetime64[ns] 323kB 2019-07-02 ... 2019-07-29T2...
  * sensor_id         (sensor_id) <U17 2kB 'DO0202_2_DO6017_4' ... 'DO1695_2_...
    length            (sensor_id) float64 184B 9.301 6.237 6.034 ... 16.77 12.43
    site_a_latitude   (sensor_id) float64 184B 51.58 51.72 51.52 ... 51.71 51.62
    site_a_longitude  (sensor_id) float64 184B 8.36 8.331 8.09 ... 8.492 8.165
    site_b_latitude   (sensor_id) float64 184B 51.66 51.66 51.57 ... 51.56 51.66
    site_b_longitude  (sensor_id) float64 184B 8.334 8.334 8.097 ... 8.545 8.334
    frequency_1       (sensor_id) float64 184B 1.954e+10 2.598e+10 ... 1.932e+10
    frequency_2       (sensor_id) float64 184B 1.852e+10 2.497e+10 ... 1.83e+10
    polarization_1    (sensor_id) <U1 92B 'V' 'V' 'V' 'V' ... 'V' 'V' 'V' 'V'
    polarization_2    (sensor_id) <U1 92B 'V' 'V' 'V' 'V' ... 'V' 'V' 'V' 'V'
Dimensions without coordinates: expert
Data variables:
    Jump              (expert, sensor_id, time) float32 15MB nan nan ... 0.0 0.0
    Dew               (expert, sensor_id, time) float32 15MB nan nan ... 0.0 0.0
    Fluctuation       (expert, sensor_id, time) float32 15MB nan nan ... 0.0 0.0
    Unknown anomaly   (expert, sensor_id, time) float32 15MB nan nan ... 0.0 0.0
    flagged           (sensor_id) bool 23B False False False ... False True
    TL_1              (sensor_id, time) float64 7MB 63.9 63.6 63.9 ... 71.8 71.5
    TL_2              (sensor_id, time) float64 7MB 64.9 64.9 64.9 ... 69.9 69.6

# Save

In [104]:
path = os.getcwd()
ds_e.to_netcdf(os.path.abspath(os.path.join(path, os.pardir))+'/example_data/cml_raw_example.nc')

/tmp/ipykernel_1533380/1455010939.py:2: SerializationWarning: saving variable Jump with floating point data as an integer dtype without any _FillValue to use for NaNs
  ds_e.to_netcdf(os.path.abspath(os.path.join(path, os.pardir))+'/example_data/cml_raw_example.nc')
/bg/fast/env_lasota-e/mambaforge/envs/tensorflow_new/lib/python3.11/site-packages/xarray/coding/variables.py:666: RuntimeWarning: invalid value encountered in cast
  data = data.astype(dtype=dtype)
/tmp/ipykernel_1533380/1455010939.py:2: SerializationWarning: saving variable Dew with floating point data as an integer dtype without any _FillValue to use for NaNs
  ds_e.to_netcdf(os.path.abspath(os.path.join(path, os.pardir))+'/example_data/cml_raw_example.nc')
/tmp/ipykernel_1533380/1455010939.py:2: SerializationWarning: saving variable Fluctuation with floating point data as an integer dtype without any _FillValue to use for NaNs
  ds_e.to_netcdf(os.path.abspath(os.path.join(path, os.pardir))+'/example_data/cml_raw_example.